# lab-10-4-mnist_nn_batchnorm
### 레이어를 지날때마다 distribution이 망가짐 : Internal Covariate Shift
### 이를 방지하는 batchnorm : 레이어의 input distribution을 normalization 시켜주어서 항상 일정하게 만들자!

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


## Load mnist

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

## Create network

In [11]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)
## dropout 없애고 batch_norm 추가
def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [12]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            ## 이 부분 수정
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.7236, train_loss: 0.34525621, train_accuracy: 0.2422, test_Accuracy: 0.1625
Epoch: [ 0] [    1/  468] time: 2.7428, train_loss: 0.32957414, train_accuracy: 0.2500, test_Accuracy: 0.2095
Epoch: [ 0] [    2/  468] time: 3.5501, train_loss: 0.29105544, train_accuracy: 0.3359, test_Accuracy: 0.3308
Epoch: [ 0] [    3/  468] time: 4.3167, train_loss: 0.24444382, train_accuracy: 0.4766, test_Accuracy: 0.4721
Epoch: [ 0] [    4/  468] time: 4.8080, train_loss: 0.32864887, train_accuracy: 0.5625, test_Accuracy: 0.5885
Epoch: [ 0] [    5/  468] time: 5.5404, train_loss: 0.25114354, train_accuracy: 0.7188, test_Accuracy: 0.6657
Epoch: [ 0] [    6/  468] time: 6.1547, train_loss: 0.23266104, train_accuracy: 0.7344, test_Accuracy: 0.7087
Epoch: [ 0] [    7/  468] time: 6.8061, train_loss: 0.28042886, train_accuracy: 0.7266, test_Accuracy: 0.7325
Epoch: [ 0] [    8/  468] time: 7.5215,

Epoch: [ 0] [   74/  468] time: 45.5149, train_loss: 0.19209853, train_accuracy: 0.9375, test_Accuracy: 0.9316
Epoch: [ 0] [   75/  468] time: 46.1328, train_loss: 0.18802410, train_accuracy: 0.9609, test_Accuracy: 0.9313
Epoch: [ 0] [   76/  468] time: 46.6870, train_loss: 0.13326585, train_accuracy: 0.9375, test_Accuracy: 0.9316
Epoch: [ 0] [   77/  468] time: 47.3260, train_loss: 0.15274850, train_accuracy: 0.9453, test_Accuracy: 0.9329
Epoch: [ 0] [   78/  468] time: 47.9081, train_loss: 0.09480438, train_accuracy: 0.9375, test_Accuracy: 0.9342
Epoch: [ 0] [   79/  468] time: 48.6645, train_loss: 0.18270651, train_accuracy: 0.9062, test_Accuracy: 0.9376
Epoch: [ 0] [   80/  468] time: 49.3395, train_loss: 0.09856392, train_accuracy: 0.9453, test_Accuracy: 0.9402
Epoch: [ 0] [   81/  468] time: 49.9979, train_loss: 0.10542075, train_accuracy: 0.9453, test_Accuracy: 0.9421
Epoch: [ 0] [   82/  468] time: 50.6411, train_loss: 0.20764408, train_accuracy: 0.9219, test_Accuracy: 0.9432
E

Epoch: [ 0] [  148/  468] time: 91.7305, train_loss: 0.13119745, train_accuracy: 0.9688, test_Accuracy: 0.9529
Epoch: [ 0] [  149/  468] time: 92.3668, train_loss: 0.09883542, train_accuracy: 0.9609, test_Accuracy: 0.9527
Epoch: [ 0] [  150/  468] time: 93.0055, train_loss: 0.08685659, train_accuracy: 0.9844, test_Accuracy: 0.9529
Epoch: [ 0] [  151/  468] time: 93.6231, train_loss: 0.14288731, train_accuracy: 0.9453, test_Accuracy: 0.9534
Epoch: [ 0] [  152/  468] time: 94.4177, train_loss: 0.12775704, train_accuracy: 0.9609, test_Accuracy: 0.9535
Epoch: [ 0] [  153/  468] time: 95.0731, train_loss: 0.10242300, train_accuracy: 0.9453, test_Accuracy: 0.9545
Epoch: [ 0] [  154/  468] time: 95.7451, train_loss: 0.09563015, train_accuracy: 0.9688, test_Accuracy: 0.9540
Epoch: [ 0] [  155/  468] time: 96.3977, train_loss: 0.07899271, train_accuracy: 0.9688, test_Accuracy: 0.9519
Epoch: [ 0] [  156/  468] time: 96.9449, train_loss: 0.11773188, train_accuracy: 0.9688, test_Accuracy: 0.9523
E

Epoch: [ 0] [  222/  468] time: 137.7220, train_loss: 0.06802465, train_accuracy: 1.0000, test_Accuracy: 0.9581
Epoch: [ 0] [  223/  468] time: 138.2682, train_loss: 0.09261975, train_accuracy: 0.9688, test_Accuracy: 0.9574
Epoch: [ 0] [  224/  468] time: 138.7575, train_loss: 0.04462041, train_accuracy: 0.9844, test_Accuracy: 0.9565
Epoch: [ 0] [  225/  468] time: 139.3147, train_loss: 0.06196169, train_accuracy: 0.9688, test_Accuracy: 0.9551
Epoch: [ 0] [  226/  468] time: 139.9718, train_loss: 0.03655334, train_accuracy: 0.9844, test_Accuracy: 0.9543
Epoch: [ 0] [  227/  468] time: 140.6328, train_loss: 0.08111545, train_accuracy: 0.9688, test_Accuracy: 0.9551
Epoch: [ 0] [  228/  468] time: 141.2339, train_loss: 0.09249762, train_accuracy: 0.9766, test_Accuracy: 0.9549
Epoch: [ 0] [  229/  468] time: 141.7018, train_loss: 0.12703034, train_accuracy: 0.9609, test_Accuracy: 0.9552
Epoch: [ 0] [  230/  468] time: 142.2931, train_loss: 0.13301958, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 183.2833, train_loss: 0.06029296, train_accuracy: 0.9922, test_Accuracy: 0.9627
Epoch: [ 0] [  297/  468] time: 183.8955, train_loss: 0.05409393, train_accuracy: 0.9766, test_Accuracy: 0.9618
Epoch: [ 0] [  298/  468] time: 184.3998, train_loss: 0.05821805, train_accuracy: 0.9531, test_Accuracy: 0.9608
Epoch: [ 0] [  299/  468] time: 184.9036, train_loss: 0.11969842, train_accuracy: 0.9766, test_Accuracy: 0.9602
Epoch: [ 0] [  300/  468] time: 185.3832, train_loss: 0.01955313, train_accuracy: 1.0000, test_Accuracy: 0.9595
Epoch: [ 0] [  301/  468] time: 185.9889, train_loss: 0.10566835, train_accuracy: 0.9766, test_Accuracy: 0.9583
Epoch: [ 0] [  302/  468] time: 186.6970, train_loss: 0.10935604, train_accuracy: 0.9609, test_Accuracy: 0.9578
Epoch: [ 0] [  303/  468] time: 187.2894, train_loss: 0.14987676, train_accuracy: 0.9688, test_Accuracy: 0.9596
Epoch: [ 0] [  304/  468] time: 187.9366, train_loss: 0.03292799, train_accuracy: 0.9922, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 228.6674, train_loss: 0.05561310, train_accuracy: 0.9688, test_Accuracy: 0.9629
Epoch: [ 0] [  371/  468] time: 229.2555, train_loss: 0.07370926, train_accuracy: 0.9766, test_Accuracy: 0.9621
Epoch: [ 0] [  372/  468] time: 229.8614, train_loss: 0.03856138, train_accuracy: 0.9844, test_Accuracy: 0.9622
Epoch: [ 0] [  373/  468] time: 230.7210, train_loss: 0.02846533, train_accuracy: 1.0000, test_Accuracy: 0.9624
Epoch: [ 0] [  374/  468] time: 231.2252, train_loss: 0.11942992, train_accuracy: 0.9609, test_Accuracy: 0.9622
Epoch: [ 0] [  375/  468] time: 231.8588, train_loss: 0.13103361, train_accuracy: 0.9453, test_Accuracy: 0.9618
Epoch: [ 0] [  376/  468] time: 232.3968, train_loss: 0.10301211, train_accuracy: 0.9844, test_Accuracy: 0.9613
Epoch: [ 0] [  377/  468] time: 233.0202, train_loss: 0.07093836, train_accuracy: 0.9609, test_Accuracy: 0.9607
Epoch: [ 0] [  378/  468] time: 233.8075, train_loss: 0.12272534, train_accuracy: 0.9453, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 272.4665, train_loss: 0.07150622, train_accuracy: 0.9688, test_Accuracy: 0.9642
Epoch: [ 0] [  445/  468] time: 273.1091, train_loss: 0.11712222, train_accuracy: 0.9531, test_Accuracy: 0.9627
Epoch: [ 0] [  446/  468] time: 273.8089, train_loss: 0.03836401, train_accuracy: 0.9844, test_Accuracy: 0.9628
Epoch: [ 0] [  447/  468] time: 274.4503, train_loss: 0.09686021, train_accuracy: 0.9609, test_Accuracy: 0.9629
Epoch: [ 0] [  448/  468] time: 275.1246, train_loss: 0.07689050, train_accuracy: 0.9609, test_Accuracy: 0.9642
Epoch: [ 0] [  449/  468] time: 275.7389, train_loss: 0.10793300, train_accuracy: 0.9609, test_Accuracy: 0.9639
Epoch: [ 0] [  450/  468] time: 276.4797, train_loss: 0.09193867, train_accuracy: 0.9531, test_Accuracy: 0.9626
Epoch: [ 0] [  451/  468] time: 277.2361, train_loss: 0.06752249, train_accuracy: 0.9844, test_Accuracy: 0.9631
Epoch: [ 0] [  452/  468] time: 277.7352, train_loss: 0.03465164, train_accuracy: 0.9922, test_Accuracy:

## Accuracy : 96.41%